# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [76]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [22]:
weather_data= pd.read_csv('weather_data.csv', encoding="utf-8")
weather_data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Rikitea,-23.12,-134.97,74.86,83,98,12.50,PF,1606267200
1,1,Goderich,43.75,-81.72,34.00,85,100,20.74,CA,1606267307
2,2,Cabadiangan,9.75,122.47,83.10,72,95,14.00,PH,1606267454
3,3,Nouadhibou,20.93,-17.03,71.60,46,87,9.17,MR,1606267264
4,4,Saldanha,-33.01,17.94,53.60,76,83,1.12,ZA,1606267455


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [23]:
#Store Lat & Lng Locations
locations= weather_data[["Lat", "Lng"]]
humidity= weather_data["Humidity"].astype(float)

In [24]:
#Plotting heat map with layer

fig = gmaps.figure()

heat_map = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=20,
                                 point_radius=1)

fig.add_layer(heat_map)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [85]:
my_cities= weather_data.loc[(weather_data["Max Temp"]>67) & (weather_data["Max Temp"]<82) & (weather_data["Cloudiness"]==0), :]
my_cities= my_cities.dropna(how='any')
my_cities.reset_index(inplace=True)
del my_cities['index']
my_cities.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,6,Mount Gambier,-37.83,140.77,73.40,46,0,16.11,AU,1606267382
1,18,Jilib,0.49,42.79,75.02,93,0,3.20,SO,1606267458
2,22,Umluj,25.02,37.27,75.42,59,0,2.98,SA,1606267459
3,28,Marsh Harbour,26.54,-77.06,73.06,76,0,22.24,BS,1606267461
4,32,Wagga Wagga,-35.12,147.37,81.00,41,0,8.05,AU,1606267461


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [89]:
#Add "Hotel Name" column to my_cities and rename to hotel_df

hotel_df= my_cities.loc[:,["City", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"]= ""
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
0,Mount Gambier,AU,-37.83,140.77,
1,Jilib,SO,0.49,42.79,
2,Umluj,SA,25.02,37.27,
3,Marsh Harbour,BS,26.54,-77.06,
4,Wagga Wagga,AU,-35.12,147.37,


In [98]:
#Store 1st hotel result into the DataFrame after locating API coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
params = {
    "location": f"{lat}, {lng}",
    "type": target_type,
    "keyword": target_keyword,
    "radius": target_radius,
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat= row['Lat']
    lng= row['Lng']
    city_name= row['City']
    
    target_type = "hotel"
    target_keyword= "hotel"
    target_radius = 5000
    
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
    except (KeyError, IndexError):
        print("Missing result...skipping")
        
    print ("-"*10)
        
print("-------End of Search-------")

Closest hotel in Mount Gambier is Federal Hotel.
----------
Closest hotel in Jilib is Federal Hotel.
----------
Closest hotel in Umluj is Federal Hotel.
----------
Closest hotel in Marsh Harbour is Federal Hotel.
----------
Closest hotel in Wagga Wagga is Federal Hotel.
----------
Closest hotel in Mataura is Federal Hotel.
----------
Closest hotel in Hithadhoo is Federal Hotel.
----------
Closest hotel in Cidreira is Federal Hotel.
----------
Closest hotel in Wau is Federal Hotel.
----------
Closest hotel in Nishihara is Federal Hotel.
----------
Closest hotel in Bolama is Federal Hotel.
----------
Closest hotel in Faanui is Federal Hotel.
----------
Closest hotel in Rocha is Federal Hotel.
----------
Closest hotel in San Juan is Federal Hotel.
----------
Closest hotel in Bonthe is Federal Hotel.
----------
Closest hotel in Sur is Federal Hotel.
----------
Closest hotel in Carnarvon is Federal Hotel.
----------
Closest hotel in Grand-Santi is Federal Hotel.
----------
Closest hotel in 

In [95]:
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
0,Mount Gambier,AU,-37.83,140.77,Federal Hotel
1,Jilib,SO,0.49,42.79,Federal Hotel
2,Umluj,SA,25.02,37.27,Federal Hotel
3,Marsh Harbour,BS,26.54,-77.06,Federal Hotel
4,Wagga Wagga,AU,-35.12,147.37,Federal Hotel


In [99]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [100]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))